# Goal: creating a mini-site using Streamlit/Dash, where you can write down your ticker and all the data will be mined automatically
+ add y/y growth to every cash-flow/balance/income
+ add average dividend yield and stuff and compare it to sector (seekingalpha.com)
+ add forecasts to plots (ARIMA or smth like that) + forecasts from analytics (seekingalpha.com)
+ add portfolio analysis (at least from etoro): sectors + dividend and price forecasting
+ add explanation to different ratios and indicators
+ add ETF analytics - sectors, top holdings (stockanalysis.com), overall position of different firms in portfolio
+ add different stuff based on alphaspread and seekingalpha examples
+ add news aggregator (maybe play a little bit with NLTK for text recognition and classification)
+ add grades from analytics and firms (current and historical, checked using logistic regression??)
+ add Monte Carlo simulation based on historical performance
+ add somewhere ML / Gradient Boosting / Decision Tree / etc. ??
+ add all of this to AWS (or other Cloud) for data to be updated automatically?

In [117]:
# finance api
import yahoofinancials
import yahooquery as yq
import yfinance as yf
import nasdaqdatalink as ndl

# data
import numpy as np
import pandas as pd
import datetime as dt

# visualization
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import streamlit as st
from plotly.subplots import make_subplots

# web scrapping
import requests
import json
import time
import re
import lxml
import cchardet
from bs4 import BeautifulSoup

NASDAQ_DATA_LINK_API_KEY = "xy8jtvPFDhiwnFktEugz" # ndl.ApiConfig.api_key 
pd.set_option('display.max_columns', None)

In [159]:
STOCK = 'MSFT'

STOCKS_LIST = ['AAPL', 'ABBV', 'ABR', 'ABT', 'AFL', 'AMD', 'AMGN', 'AMZN', 
    'ASML', 'ATVI', 'AVGO', 'BBY', 'BHP', 'BRK-B', 'CCI',
    'COST', 'CSCO', 'CVX', 'GOOGL', 'HD', 'IRM', 'JNJ', 'JPM',
    'LOW', 'MA', 'MAIN', 'MCD', 'MCO', 'MDT', 'META', 'MRK',
    'MS', 'MSFT', 'NFLX', 'NVDA', 'O', 'PEP', 'PFE', 'PLD', 'PYPL',
    'SBUX', 'STOR', 'TGT', 'TROW', 'TSLA',
    'TSM', 'UNP', 'UPS', 'V', 'VICI', 'WPC', 'XOM', 'ZTS']

CRYPTO_LIST = ['BTC-USD', 'ETH-USD']

ETF_LIST = ['SCHD', 'SPHD', 'VOO', 'QQQ', 'VGT', 'ARKK']

WATCHLIST_LIST = ['MO', 'T']

In [160]:
stock_list_new = [x.lower() if x != 'BRK-B' else 'brk.b' for x in STOCKS_LIST]
stock_list_test = [STOCK]

In [161]:
def get_macrotrends_data(url:str):
    response = requests.get(url)
    html = response.text
    soup = BeautifulSoup(html, "html.parser")
    scripts = soup.find_all("script", type="text/javascript")

    for script in scripts:
        if 'originalData' in script.text:
            var_data = script.text

    # Use regular expressions to extract the variable
    match = re.search('var originalData = (.*);', var_data)
    data = match.group(1)
    df = pd.read_json(data)

    for i, k in enumerate(df['field_name']):
        if BeautifulSoup(k, "html.parser").find('a'):
            df.loc[i, "field_name"] = BeautifulSoup(k, "html.parser").find('a').text
        else:
            df.loc[i, "field_name"] = BeautifulSoup(k, "html.parser").find('span').text

    df = df.drop(columns=['popup_icon']).set_index('field_name').replace('', 0).astype(float).T
    return df

In [205]:
def create_plot_bar_line(df:pd.DataFrame, bar='', line='', y2perc=False, secondary_y=True, bar_color='#5a7d9f', line_color='white', title=''):
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    fig.add_bar(x=df.index, y=df[bar], name=bar, secondary_y=False, marker_color=bar_color)

    if title !='':
        title_ = title
    else:
        title_ = bar
        
    fig.update_layout(height=400, width=600, margin=dict(l=20, r=20, t=30, b=20), template="plotly_dark", title=title_,
        legend=dict(orientation="h", yanchor="bottom", y = -0.15, xanchor="right", x = 0.75),
    )

    if line != '':
        fig.add_scatter(x=df.index, y=df[line], name=line, secondary_y=secondary_y, marker_color=line_color)
        if y2perc:
            fig.update_layout(yaxis2=dict(tickformat='.0%'))

    # fig.show()
    
    return fig

In [163]:
def create_ema_plot(tickers_list, start_date='2019-01-01', period='days', emas=[1]):

    colors={'standard':'#424242', 
        'min':'#00ab41', 
        'other':'#8B8000', 
        'max':'#c30010'}
    
    df_orig = pd.DataFrame(yf.download(tickers_list, start_date)['Adj Close'])

    if len(tickers_list)>1:
        vertical_spacing = (1 / (len(tickers_list) - 1))/9
    else:
        vertical_spacing=0
        df_orig.columns = tickers_list

    df = df_orig.copy()
    
    for n in emas:
        ema_period = f"{n} {period}"
        ema_df = df_orig.ewm(halflife=ema_period, times=df_orig.index, ignore_na=False).mean()
        ema_df.columns = [f"{c}_{n}{period}_ema" for c in ema_df.columns]
        df = pd.concat([df, ema_df], axis=1)

    fig = make_subplots(
        rows=len(tickers_list),
        cols=1,
        subplot_titles=tickers_list,
        vertical_spacing=vertical_spacing)
    
    for i, c in enumerate(tickers_list, start=1):
        df_columns = [col for col in df.columns if col.startswith(c+'_')|(col==c)]
        df_new = df[df_columns].copy()
        for i2, c2 in enumerate(df_columns):

            if i2 == 0:
                color = colors['standard']
                width = 2
            elif i2 == 1:
                color = colors['min']
                width = 2
            elif i2 == len(df_columns)-1:
                color = colors['max']
                width = 2
            else:
                color = colors['other']
                width = 0.5
            
            fig.add_trace(
                go.Scatter(
                    x=df_new.index, 
                    y=df_new[c2], 
                    line=dict(color=color, width=width), 
                    name=c2
                ), 
                row=i, 
                col=1
            )

    fig.update_layout(
        height=len(tickers_list)*400, 
        width=600, 
        showlegend=False, 
        margin=dict(l=20, r=20, t=30, b=20),
        template="plotly_dark"
    )

    fig.show()
    
    return df

In [164]:
def create_line_plot(df:pd.DataFrame, y:list, title='', perc=True):
    fig = go.Figure()
    
    for line in y:
        fig.add_scatter(
            x=df.index, 
            y=df[line], 
            name=line
        )

    fig.update_layout(
        height=400, 
        width=600, 
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y = -0.25,
            xanchor="right",
            x = 0.75
        ),
        margin=dict(l=20, r=20, t=30, b=20),
        template="plotly_dark",
        title=title
    )

    if perc:
        fig.update_layout(yaxis=dict(tickformat='.1%'))

    fig.show()
    return fig

# API

Dividend data

In [165]:
"""Seeking alpha full dividend history"""

div_history_df = pd.DataFrame()

for tick in stock_list_test:
    url = f"https://seekingalpha.com/api/v3/symbols/{tick.lower()}/dividend_history"
    querystring = {"years":"100"}
    headers = {
        "cookie": "machine_cookie=4979826528810; _cls_v=072cd8fc-83ec-4b6d-b840-72ce92a351d4; _cls_s=da78f999-6e82-4412-bfd3-98a35379d96d:0; _pxvid=6190f403-0540-11ed-8356-71796f6e5767; pxcts=61910480-0540-11ed-8356-71796f6e5767; g_state=^{^\^i_l^^:0^}; has_paid_subscription=false; OptanonAlertBoxClosed=2022-07-16T19:49:37.138Z; _ga=GA1.2.422884809.1658000977; _igt=80f0662b-29d6-4ba2-daef-f15a084be986; _hjSessionUser_65666=eyJpZCI6IjVmNjA3NTU1LTFmODItNWFhOC05NzBkLTMxNmIwOTFkNDJjZSIsImNyZWF0ZWQiOjE2NTgwNDMwMjQxNTYsImV4aXN0aW5nIjp0cnVlfQ==; _hjCachedUserAttributes=eyJhdHRyaWJ1dGVzIjp7ImxvZ2dlZF9pbiI6dHJ1ZSwibXBfc3ViIjpmYWxzZSwicHJlbWl1bV9zdWIiOmZhbHNlLCJwcm9fc3ViIjpmYWxzZX0sInVzZXJJZCI6IjU2ODczOTA0In0=; ga_clientid=422884809.1658000977; _pcid=^%^7B^%^22browserId^%^22^%^3A^%^22l6l1zvh16ggo2rl5^%^22^%^7D; _clck=1sv21qj^|1^|f4c^|0; _ig=56873904; sailthru_content=2528dc295dc3fbbf1ec8e71fd6af16ea5ed0fab1751712d30b586234ac21ac69c6f48017810681510ac670347a1b237b395addcc8a084ec17e397065464a467803e85c27969d6ca11adf1e5bae9ce43e365ade53ba1716e0f5409199ca81b1b2d336ff2bdab2770099e746360c3b2e4a8f46c8cbd3b263891ad28c66986af90e8a2bb0fb3446957f12521164830063aa9eada221935b05aaed9d45ccc5957509; sailthru_visitor=4a85db3b-194e-42bd-bc87-31076f836304; sailthru_hid=29f91ce2c0119534955a4934eea65d5d62d3164919e4cd8e5507453023d2712d74fca4d95585b51117583622; _gcl_au=1.1.905016176.1671643238; __pat=-18000000; user_id=56873904; user_nick=; user_devices=2; u_voc=; marketplace_author_slugs=; user_cookie_key=cjjdiz; user_perm=; sapu=101; user_remember_token=04b7dcb2602e3f78db1c7c7b3e0e43599aa202f5; _sapi_session_id=0pCP6BL7ckaTjzz1yGfnvj2fYymMCVyRcdc0FilJJuJrLs^%^2BPk6M7pmkTNZq^%^2Bs0tQzLw0Gwxfpuz4XXdeLwjnEvGdwVGKVQdIhiI4kf6GgA6c6Aqo8EAHDVX3JUirUkOfv7^%^2Fv6zuUolHyz^%^2Bka3l7tx2Tmr6LfeaHe0syKkJJ99iSM^%^2FbcPrEEdST3wciFuUBwzxt3V9trL98gAlWdoY4Ces0hsdCU^%^2BEryApHpHc9rt8S2ZjmXsQ7PNxkHufEwIxhqC2LmTKsoVyrOgYz4rWUiq8CGM^%^2BdxILxHnEzl1LN9h2hU^%^3D--^%^2Fq^%^2FbqzYaui40jz7x--I^%^2FfbuLyN7DqYI^%^2BHocBaR9A^%^3D^%^3D; _pctx=^%^7Bu^%^7DN4IgrgzgpgThIC5QFYBsAOA7AZgJwAYAWRUABxigDMBLAD0RBABoQAXAT1KgYDUANEAF9BLSLADKrAIatIDCgHNqEVrCgATZiAjVVASU0IAdmAA2pwUA; _pxhd=9b81b7053d831d0e418b92698dce0fc88c8297e1e67eb88e98fefc26b9d3b6ac:80650f60-6b3b-11e9-814e-41aaaa844f02; ubvt=b26b3487-0e8c-451d-9656-705df157b6a2; session_id=27a89810-0094-4454-8793-f52f76340fbd; OptanonConsent=isIABGlobal=false&datestamp=Thu+Dec+22+2022+16^%^3A05^%^3A26+GMT^%^2B0100+(czas+^%^C5^%^9Brodkowoeuropejski+standardowy)&version=6.30.0&landingPath=NotLandingPage&groups=C0001^%^3A1^%^2CC0002^%^3A1^%^2CC0003^%^3A1^%^2CC0007^%^3A1&hosts=H40^%^3A1^%^2CH17^%^3A1^%^2CH13^%^3A1^%^2CH36^%^3A1^%^2CH55^%^3A1^%^2CH69^%^3A1^%^2CH45^%^3A1^%^2CH14^%^3A1^%^2CH15^%^3A1^%^2CH19^%^3A1^%^2CH47^%^3A1&AwaitingReconsent=false&genVendors=V12^%^3A1^%^2CV5^%^3A1^%^2CV7^%^3A1^%^2CV8^%^3A1^%^2CV13^%^3A1^%^2CV15^%^3A1^%^2CV3^%^3A1^%^2CV2^%^3A1^%^2CV6^%^3A1^%^2CV14^%^3A1^%^2CV1^%^3A1^%^2CV4^%^3A1^%^2CV9^%^3A1^%^2C&geolocation=PL^%^3B14; __pnahc=1; gk_user_access=1**1671790151; gk_user_access_sign=316999477f1cf3b270ec2daee33355ef077c23cf; __tac=; __tae=1671790157992; LAST_VISITED_PAGE=^%^7B^%^22pathname^%^22^%^3A^%^22https^%^3A^%^2F^%^2Fseekingalpha.com^%^2Fsymbol^%^2FDPZ^%^2Fdividends^%^2Fhistory^%^22^%^2C^%^22pageKey^%^22^%^3A^%^22ba85820c-c9a7-4301-91ed-047be2dec0c2^%^22^%^7D; _uetsid=c9555410815311ed8383e1bd89176270; _uetvid=6c9a7a40054011ed9912e34a5318d584; __pvi=eyJpZCI6InYtMjAyMi0xMi0yMy0xMS0wOS0xNC0zMDYtRFVlQXM1NWtGcHdFelhldy05OWVlM2VhYmJkMDU0N2NiMjRiMjQ2ZTU5ZTc4YmQ4OCIsImRvbWFpbiI6Ii5zZWVraW5nYWxwaGEuY29tIiwidGltZSI6MTY3MTc5MDY2ODc4NH0^%^3D; __tbc=^%^7Bkpex^%^7Dc34b4dUSkelinBilgVjlXAFjdExL2yDTVVsaH2tHeWieSgu52a503DdkAZX5En4R; xbc=^%^7Bkpex^%^7DpsZvcg-czvsWNhuvqvMZK8J5UpYhUPaAf31G9LNO4s_JNybiiLibHlVRHn3hm4E4nn-OgFei0KNGMmPkAUA1_w-h83kuroSVs6Wm4u7Ywo2khMWDgt1X4fFsw_eRSpv_RT073ml6wbguc-BKt5xBC3jze6MTqMhOTtHPaQlo8jgrWISTUeJdpSW5wg1k8whSzoS5_JJNFGD12hP_7LIJ9Rcboio5C_pfp4SlYIgOvl0t0F4JUlwH3AItmjnB36P2lQd46Wi4gj8SrJp-WVo44vskLuAbTmezh-9Nmb6v2dAtnefy1d_SnhK1ucoCCPyx9eHnXkzHTxLTKoa4V1CaJBGXBFnLuyNvM48L074T6SRARQTZyVNljtYreNy7Uxb-agK4V0R54vP3iIc0NEPleFizxGh8FZZoF4flQb7mGezf-1HBFpWUlIR7p55GktmivP2SWPpXI1SzKXApvhhYN_mlYAm6eHG7Pq1LZgIR4zWUkv2RKy3rJd9Qsk8cHLPlvjhuRmx_t1ZjQa7IsxW7_03FS_lF67VC3PfVw_sI7vJlVj9ccU7hT9ptOtwx7ECKKYPkv5zP7q_a3Yubi4CmIM5MP-cJhy_-6RU96KhQ-FqXxVYETn_nJbtT3MXgwQma1soxbODUZ0d9NKNDWU5_lu9l2WXp88Vf-PdLt9LNv-Q",
        "authority": "seekingalpha.com",
        "referer": f"https://seekingalpha.com/symbol/{tick}/dividends/history",
        "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36"
    }
    response = requests.request("GET", url, headers=headers, params=querystring).json()['data']

    for id in response:
        row = pd.DataFrame([id['attributes']])
        row['Ticker'] = tick
        div_history_df = pd.concat([div_history_df, row], axis=0)

    if len(stock_list_test) >1:
        time.sleep(2) # in case of generating data on more then 1 ticker

for c in [c for c in div_history_df.columns if '_date' in c]:
    div_history_df[c] = pd.to_datetime(div_history_df[c])
div_history_df['amount'] = div_history_df['amount'].astype(float)
div_history_df = div_history_df.reset_index(drop=True).set_index('date')

In [166]:
"""NASDAQ dividend data"""

url = f"https://api.nasdaq.com/api/quote/{STOCK}/dividends"
querystring = {"assetclass":"stocks"}

headers = {
    "authority": "api.nasdaq.com",
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36"
}
response = requests.request("GET", url, headers=headers, params=querystring).json()

ex_div_date = response['data']['exDividendDate']
div_pay_date = response['data']['dividendPaymentDate']
div_yield = response['data']['yield']
annualized_div = response['data']['annualizedDividend']

# dividend_history = pd.DataFrame(response['data']['dividends']['rows'])

# for c in [c for c in dividend_history.columns if 'Date' in c]:
#     dividend_history[c] = pd.to_datetime(dividend_history[c])
# dividend_history = dividend_history.set_index('recordDate')
# dividend_history['amount'] = dividend_history['amount'].str.replace('$', '').astype(float)

# create_plot_bar_line(dividend_history, 'amount', title='NASDAQ dividend history')

print(f'ex_div_date: {ex_div_date}')
print(f'div_pay_date: {div_pay_date}')
print(f'div_yield: {div_yield}')
print(f'annualized_div: {annualized_div}')

ex_div_date: 11/16/2022
div_pay_date: 12/08/2022
div_yield: 1.15%
annualized_div: 2.72


StockAnalysis Financials

In [167]:
period = 'quarterly'
income_statements = pd.DataFrame()

for x in stock_list_test: # STOCKS_LIST
    print(x)

    url = f"https://stockanalysis.com/stocks/{x.lower()}/financials/__data.json"

    querystring = {"x-sveltekit-invalidated":"__1"} # also period can be "trailing" or no period at all (annual)
    if (period == 'quarterly')|(period == 'trailing'):
        querystring['period'] = period

    headers = {
        "authority": "stockanalysis.com",
        "accept": "*/*",
        "accept-language": "en-US,en;q=0.9,ru-RU;q=0.8,ru;q=0.7,uk-UA;q=0.6,uk;q=0.5,pl;q=0.4",
        "cookie": "cf_chl_2=9c80dd02f1fc73c; cf_clearance=OwRTeLsjteKSq2vOGA415U77v0RksWzpM_0xtiixnIA-1671821196-0-160",
        "referer": f"https://stockanalysis.com/stocks/{x.lower()}/financials/",
        "sec-ch-ua": "^\^Not?A_Brand^^;v=^\^8^^, ^\^Chromium^^;v=^\^108^^, ^\^Google",
        "sec-ch-ua-mobile": "?0",
        "sec-ch-ua-platform": "^\^Windows^^",
        "sec-fetch-dest": "empty",
        "sec-fetch-mode": "cors",
        "sec-fetch-site": "same-origin",
        "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36"
    }
    
    response = requests.request("GET", url, headers=headers, params=querystring).json()

    # with open("stock_analysis.json", "w") as outfile:
    #     json.dump(response, outfile)

    response_new = response['nodes']
    for i in response_new:
        if i['type'] != 'skip':
            data = list(i['data'])
    
    dict1 = {}
    for k, v in zip(data[0].keys(), data[0].values()):
        dict1[k] = data[v]

    dict2 = {}
    for k, v in zip(dict1['financialData'].keys(), dict1['financialData'].values()):
        dict2[k] = data[v]

    dict3 = {}
    for k, v_list in zip(dict2.keys(), dict2.values()):
        values = []
        for v in v_list:
            values = values + [data[v]]
        dict3[k] = values

    dict_names = {}
    for m in dict1['map']:
        dict_names[data[data[m]['id']]] = data[data[m]['title']]
    dict_names['fcf'] = 'Free Cash Flow'
    dict_names['datekey'] = 'Date'

    income_df = pd.DataFrame(dict3)
    income_df['Ticker'] = x
    income_df = income_df.rename(columns=dict_names)
    
    income_statements = pd.concat([income_statements, income_df], axis=0)

    # time.sleep(4) # in case of generating data on more then 1 ticker

income_statements = income_statements.set_index('Date')
income_statements['Dividends'] = income_statements['Dividend Per Share'] * income_statements['Shares Outstanding (Basic)']
income_statements.head(5)

MSFT


,Revenue,Revenue Growth (YoY),Cost of Revenue,Gross Profit,"Selling, General & Admin",Research & Development,Operating Expenses,Other Operating Expenses,Operating Income,Interest Expense / Income,Other Expense / Income,Pretax Income,Income Tax,Preferred Dividends,Net Income,Net Income Growth,Shares Outstanding (Basic),Shares Outstanding (Diluted),Shares Change,EPS (Basic),EPS (Diluted),EPS Growth,Free Cash Flow Per Share,Dividend Per Share,Dividend Growth,Free Cash Flow,Gross Margin,Operating Margin,Profit Margin,Free Cash Flow Margin,Effective Tax Rate,EBITDA,EBITDA Margin,Depreciation & Amortization,EBIT,EBIT Margin,Ticker,Dividends
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2022-09-30,50122000000,0.106031,15452000000,34670000000,6524000000,6628000000,13152000000,0,21518000000,500000000,-554000000,21572000000,4016000000,0,17556000000,-0.143819,7457000000,7485000000,-0.010837,2.35,2.35,-0.132841,2.268,0.62,0.107143,16915000000,0.691712,0.429312,0.350265,0.337477,0.186167,24862000000,0.496030,2790000000,22072000000,0.440366,MSFT,4.623340e+09
2022-06-30,51865000000,0.123787,16429000000,35436000000,8053000000,6849000000,14902000000,0,20534000000,496000000,-449000000,20487000000,3747000000,0,16740000000,0.017135,7472000000,7504000000,-0.010157,2.24,2.24,0.032258,2.377,0.62,0.107143,17758000000,0.683235,0.395912,0.322761,0.342389,0.182896,24962000000,0.481288,3979000000,20983000000,0.404570,MSFT,4.632640e+09
2022-03-31,49360000000,0.183523,15615000000,33745000000,7075000000,6306000000,13381000000,0,20364000000,503000000,-329000000,20190000000,3462000000,0,16728000000,0.082228,7493000000,7534000000,-0.008293,2.23,2.22,0.093596,2.675,0.62,0.107143,20046000000,0.683651,0.412561,0.338898,0.406118,0.171471,24466000000,0.495665,3773000000,20693000000,0.419226,MSFT,4.645660e+09
2021-12-31,51728000000,0.200854,16960000000,34768000000,6763000000,5758000000,12521000000,0,22247000000,525000000,-793000000,22515000000,3750000000,0,18765000000,0.213542,7505000000,7555000000,-0.008009,2.50,2.48,0.221675,1.148,0.62,0.107143,8615000000,0.672131,0.430077,0.362763,0.166544,0.166556,26536000000,0.512991,3496000000,23040000000,0.445407,MSFT,4.653100e+09
2021-09-30,45317000000,0.219707,13646000000,31671000000,5834000000,5599000000,11433000000,0,20238000000,539000000,-825000000,20524000000,19000000,0,20505000000,0.475923,7513000000,7567000000,-0.009166,2.73,2.71,0.489011,2.493,0.56,0.098039,18730000000,0.698877,0.446587,0.452479,0.413311,0.000926,24275000000,0.535671,3212000000,21063000000,0.464792,MSFT,4.207280e+09


In [168]:
# expenses dataframe for plot
expenses = [
    'Selling & Marketing',
    'General & Administrative',
    'Selling, General & Admin',
    'Research & Development',
    'Other Operating Expenses',
    "Interest Expense",
    'Interest Expense / Income',
    'Other Expense / Income'
    ]
df_expenses = income_statements.loc[:,[c for c in income_statements.columns if c in expenses]]
for c in df_expenses.columns:
    if 'Expense / Income' in c:
        for i in df_expenses.index:
            if df_expenses.loc[i,c]>0:
                df_expenses.loc[i,c] = 0
            else:
                df_expenses.loc[i,c] = -df_expenses.loc[i,c]
df_expenses.head(5)

,"Selling, General & Admin",Research & Development,Other Operating Expenses,Interest Expense / Income,Other Expense / Income
Date,,,,,
2022-09-30,6524000000,6628000000,0,0,554000000
2022-06-30,8053000000,6849000000,0,0,449000000
2022-03-31,7075000000,6306000000,0,0,329000000
2021-12-31,6763000000,5758000000,0,0,793000000
2021-09-30,5834000000,5599000000,0,0,825000000


In [169]:
revenue_plot = create_plot_bar_line(income_statements, 'Revenue', 'Net Income', secondary_y=False) # Cost of Revenue
eps_plot = create_plot_bar_line(income_statements, 'EPS (Basic)', 'EPS Growth', y2perc=True, bar_color='#7eb37a')
ebitda_plot = create_plot_bar_line(income_statements, 'EBITDA', 'EBITDA Margin', y2perc=True, bar_color='#805d67')
shares_plot = create_plot_bar_line(income_statements, 'Shares Outstanding (Basic)', 'Shares Change', y2perc=True, bar_color='#ea7726') 
fcf_plot = create_plot_bar_line(income_statements, 'Free Cash Flow', 'Free Cash Flow Per Share', bar_color='#8d8b55')
profit_plot = create_plot_bar_line(income_statements, 'Gross Profit', 'Operating Expenses', secondary_y=False, bar_color='#7c459c')
dividends_plot = create_plot_bar_line(income_statements, 'Dividend Per Share', 'Dividend Growth', y2perc=True, bar_color='#03c03c')
# dividends_full_history = create_plot_bar_line(div_history_df, 'amount', 'adjusted_amount', title='Full dividend history', bar_color='#03c03c')
margins_plot = create_line_plot(income_statements, y=['Gross Margin', 'Operating Margin', 'Profit Margin', 'Free Cash Flow Margin'], title='Margins')
ema_plot = create_ema_plot([STOCK], emas=[10, 20, 30, 40, 50])

[*********************100%***********************]  1 of 1 completed


In [198]:
# crypto_df = create_ema_plot(CRYPTO_LIST, emas=[10, 20, 30, 40, 50])
# stocks_df = create_ema_plot(STOCKS_LIST, emas=[10, 20, 30, 40, 50])
def create_stacker_bar(df:pd.DataFrame, title_ ='', colors=px.colors.sequential.Viridis):
    stacked_plot = px.bar(data_frame=df, color_discrete_sequence=colors)
    stacked_plot.update_layout(
        xaxis = dict(title=None),
        yaxis = dict(title=None),
        height=400, 
        width=600, 
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y = -0.4,
            xanchor="left",
            title= None
        ),
        margin=dict(l=20, r=20, t=30, b=20),
        template="plotly_dark",
        title=title_
    )
    return stacked_plot

expenses_plot = create_stacker_bar(df_expenses, title_='Expenses', colors=px.colors.sequential.Turbo_r)
expenses_plot

macrotrends

In [130]:
tickers_macrotrends_dict = {}
macrotrends_list = requests.get("https://www.macrotrends.net/assets/php/ticker_search_list.php?_=1673472383864").json()

for e in macrotrends_list:
    url_link = list(e.values())[1]
    ticker = list(e.values())[0].split(' - ')[0]
    tickers_macrotrends_dict[ticker] = url_link
# tickers_macrotrends_dict

In [172]:
# balance
balance = get_macrotrends_data(f"https://www.macrotrends.net/stocks/charts/{tickers_macrotrends_dict[STOCK]}/balance-sheet?freq=Q") # ?freq=A
balance.head()

field_name,Cash On Hand,Receivables,Inventory,Pre-Paid Expenses,Other Current Assets,Total Current Assets,"Property, Plant, And Equipment",Long-Term Investments,Goodwill And Intangible Assets,Other Long-Term Assets,Total Long-Term Assets,Total Assets,Total Current Liabilities,Long Term Debt,Other Non-Current Liabilities,Total Long Term Liabilities,Total Liabilities,Common Stock Net,Retained Earnings (Accumulated Deficit),Comprehensive Income,Other Share Holders Equity,Share Holder Equity,Total Liabilities And Share Holders Equity
2022-09-30,107262.0,31279.0,4268.0,0.0,18003.0,160812.0,77037.0,6839.0,78267.0,23482.0,198972.0,359784.0,87389.0,45374.0,39023.0,98829.0,186218.0,88535.0,92374.0,-7343.0,0.0,173566.0,359784.0
2022-06-30,104757.0,44261.0,3742.0,0.0,16924.0,169684.0,74398.0,6891.0,78822.0,21897.0,195156.0,364840.0,95082.0,47032.0,41595.0,103216.0,198298.0,86939.0,84281.0,-4678.0,0.0,166542.0,364840.0
2022-03-31,104693.0,32613.0,3296.0,0.0,13320.0,153922.0,70298.0,6907.0,78719.0,21845.0,190685.0,344607.0,77439.0,48177.0,41637.0,104244.0,181683.0,85767.0,79633.0,-2476.0,0.0,162924.0,344607.0
2021-12-31,125369.0,33520.0,3019.0,0.0,12280.0,174188.0,67214.0,6994.0,58383.0,21256.0,166201.0,340389.0,77510.0,48260.0,40868.0,102869.0,180379.0,84528.0,75045.0,437.0,0.0,160010.0,340389.0
2021-09-30,130615.0,27349.0,3411.0,0.0,12951.0,174326.0,63772.0,6393.0,58249.0,21103.0,161092.0,335418.0,80528.0,50039.0,40061.0,102912.0,183440.0,83751.0,66944.0,1283.0,0.0,151978.0,335418.0


In [201]:
assets_stackplot = create_stacker_bar(balance[['Cash On Hand',
    'Receivables',
    'Inventory',
    'Pre-Paid Expenses',
    'Other Current Assets',
    'Property, Plant, And Equipment',
    'Long-Term Investments',
    'Goodwill And Intangible Assets',
    'Other Long-Term Assets'
    ]], title_='Total assets')
assets_stackplot

In [200]:
liabilities_stackplot = create_stacker_bar(balance[[
    'Total Current Liabilities'
    ,'Long Term Debt'
    ,'Other Non-Current Liabilities'
    ,'Total Long Term Liabilities'
    ,'Common Stock Net'
    ,'Retained Earnings (Accumulated Deficit)'
    ,'Comprehensive Income'
    ,'Other Share Holders Equity'
    ]], title_='Total liabilities')
liabilities_stackplot

In [202]:
# income statement
# income_df = get_macrotrends_data(f"https://www.macrotrends.net/stocks/charts/{tickers_macrotrends_dict[STOCK]}/income-statement?freq=Q")
# income_df.head()

In [133]:
# cash flow statement
cash_flow_df = get_macrotrends_data(f"https://www.macrotrends.net/stocks/charts/{tickers_macrotrends_dict[STOCK]}/cash-flow-statement?freq=Q")
cash_flow_df.head()

field_name,Net Income/Loss,Total Depreciation And Amortization - Cash Flow,Other Non-Cash Items,Total Non-Cash Items,Change In Accounts Receivable,Change In Inventories,Change In Accounts Payable,Change In Assets/Liabilities,Total Change In Assets/Liabilities,Cash Flow From Operating Activities,"Net Change In Property, Plant, And Equipment",Net Change In Intangible Assets,Net Acquisitions/Divestitures,Net Change In Short-term Investments,Net Change In Long-Term Investments,Net Change In Investments - Total,Investing Activities - Other,Cash Flow From Investing Activities,Net Long-Term Debt,Net Current Debt,Debt Issuance/Retirement Net - Total,Net Common Equity Issued/Repurchased,Net Total Equity Issued/Repurchased,Total Common And Preferred Stock Dividends Paid,Financial Activities - Other,Cash Flow From Financial Activities,Net Cash Flow,Stock-Based Compensation,Common Stock Dividends Paid
2022-09-30,336.878,5.106,129.239,134.3449,0.0,0.0,0.0,33.120,33.120,504.343,-0.472,0.0,0.0,-207.722,1.080,-206.642,-39.628,-246.742,600.0,-600.0,0.0,0.000,0.000,-346.518,-6.701,-353.219,-95.618,3.493,-346.518
2022-06-30,-58.141,8.611,680.004,688.6150,0.0,0.0,0.0,22.487,22.487,652.961,-0.232,0.0,0.0,0.000,4012.146,4012.146,-8626.025,-4614.111,4400.0,0.0,4400.0,-3.428,-3.428,-269.325,-120.798,4006.449,45.299,3.236,-269.325
2022-03-31,242.688,16.753,47.886,64.6390,0.0,0.0,0.0,-9.154,-9.154,298.173,-0.454,0.0,0.0,0.000,-4012.096,-4012.096,-15.695,-4028.245,600.0,0.0,600.0,3216.411,3216.411,-227.863,-29.388,3559.160,-170.912,2.630,-227.863
2021-12-31,283.798,21.500,-23.052,-1.5520,0.0,0.0,0.0,91.307,3.180,285.426,8.636,0.0,0.0,0.000,40.000,40.000,-27.043,21.593,0.0,0.0,0.0,-0.437,-0.437,-226.430,-10.052,-236.919,70.100,2.304,-226.430
2021-09-30,164.184,34.869,-4.377,30.4920,0.0,0.0,0.0,5.778,5.778,200.454,3.683,0.0,0.0,0.000,0.000,0.000,-25.932,-22.249,-2100.0,0.0,-2100.0,2386.129,2386.129,-177.104,-25.238,83.787,261.992,2.395,-177.104


In [206]:
compensation_plot = create_plot_bar_line(cash_flow_df, 'Stock-Based Compensation', bar_color="#EADA52")
compensation_plot

In [134]:
# financial ratios
fin_ratios_df = get_macrotrends_data(f"https://www.macrotrends.net/stocks/charts/{tickers_macrotrends_dict[STOCK]}/financial-ratios?freq=Q")
fin_ratios_df.head()

field_name,Current Ratio,Long-term Debt / Capital,Debt/Equity Ratio,Gross Margin,Operating Margin,EBIT Margin,EBITDA Margin,Pre-Tax Profit Margin,Net Profit Margin,Asset Turnover,Inventory Turnover Ratio,Receiveable Turnover,Days Sales In Receivables,ROE - Return On Equity,Return On Tangible Equity,ROA - Return On Assets,ROI - Return On Investment,Book Value Per Share,Operating Cash Flow Per Share,Free Cash Flow Per Share
2022-09-30,0.0,0.3836,0.6224,0.0,63.9892,63.9892,0.0,44.8803,44.0301,0.0201,0.0,0.0,0.0,1.5269,1.5269,0.9030,0.9412,22.9078,0.4487,0.4483
2022-06-30,0.0,0.3829,0.6205,0.0,9.0700,9.0700,0.0,-8.6194,-8.7087,0.0178,0.0,0.0,0.0,-0.2629,-0.2629,-0.1559,-0.1622,22.9624,0.6275,0.6273
2022-03-31,0.0,0.2549,0.3421,0.0,74.6803,74.6803,0.0,58.3470,57.6977,0.0192,0.0,0.0,0.0,1.5674,1.5674,1.1163,1.1678,20.6888,0.4334,0.4328
2021-12-31,0.0,0.2781,0.3852,0.0,92.6387,92.6387,0.0,74.2670,73.4637,0.0218,0.0,0.0,0.0,2.3287,2.3287,1.6127,1.6811,19.3773,0.4851,0.5000
2021-09-30,0.0,0.2789,0.3869,0.0,91.8987,91.8987,0.0,43.8036,43.0823,0.0214,0.0,0.0,0.0,43.6736,1.3537,0.9356,0.9761,19.2843,0.3279,0.3344


Seeking Alpha

In [135]:
div_list_seeking_alpha = [
    'div_growth_category_avg_5y',
    'dps_yoy_avg_5y',
    'dividend_per_share_change_dislpay_avg_5y',
    'dividend_lt_fwd_growth_avg_5y',
    'div_grow_rate3_avg_5y',
    'div_grow_rate5_avg_5y',
    'div_grow_rate10_avg_5y',
    'revenue_change_display_avg_5y',
    'eps_change_display_avg_5y',
    'fcf_per_share_change_display_avg_5y',
    'ebitda_change_display_avg_5y',
    'ebit_change_display_avg_5y',
    'return_on_net_tangible_assets_avg_5y',
    'log_of_unadjusted_stock_price_avg_5y',
    'coefficient_of_variation_90d_avg_5y',
    'degree_of_operating_leverage_ttm_avg_5y',
    'div_growth_category',
    'dps_yoy',
    'div_rate',
    'dividend_per_share_change_dislpay',
    'dividend_lt_fwd_growth',
    'div_grow_rate3',
    'div_grow_rate5',
    'div_grow_rate10',
    'revenue_change_display',
    'eps_change_display',
    'fcf_per_share_change_display',
    'ebitda_change_display',
    'ebit_change_display',
    'return_on_net_tangible_assets',
    'log_of_unadjusted_stock_price',
    'coefficient_of_variation_90d',
    'degree_of_operating_leverage_ttm',
    'div_yield_category',
    'div_yield_4y',
    'dividend_yield',
    'div_yield_fwd',
    'yld_on_cost_1y',
    'yld_on_cost_3y',
    'yld_on_cost_5y',
    'earnings_yield',
    'earn_yield_gaap_fy1',
    'oper_income_market_cap',
    'oper_income_fy1_market_cap',
    'fcf_yield',
    'fcf_yield_fy1'
]

In [136]:
# div_growth
url = "https://seekingalpha.com/api/v3/metrics"
querystring = {
    "filter[fields][]":div_list_seeking_alpha,
    "filter[slugs]":f"{STOCK.lower()}",
    "minified":"false"
}
headers_div_growth = {
    "cookie": "machine_cookie=9717268612629; machine_cookie_ts=1671790378",
    "authority": "seekingalpha.com",
    "referer": f"https://seekingalpha.com/symbol/{STOCK}/dividends/dividend-growth",
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36"
}
response = requests.request("GET", url, headers=headers_div_growth, params=querystring).json()
result = {}
for item in response["data"]:
    field = next(filter(lambda included: included["id"] == item["relationships"]["metric_type"]["data"]["id"], response["included"]))["attributes"]["field"]
    result[field] = item["attributes"]["value"]
result

{'revenue_change_display': 37.812984,
 'ebit_change_display': 50.25748,
 'eps_change_display': 10.779042,
 'dividend_yield': 4.409171,
 'fcf_per_share_change_display': 38.359123,
 'ebitda_change_display': 35.814266,
 'div_yield_4y': 4.948753,
 'div_yield_fwd': 4.585538,
 'dividend_per_share_change_dislpay': 8.41402,
 'dividend_lt_fwd_growth': 4.213091,
 'div_grow_rate3': 8.634674084667404,
 'dps_yoy': 8.8888,
 'earnings_yield': 3.914609,
 'oper_income_market_cap': 3.645994,
 'oper_income_fy1_market_cap': 4.872186,
 'fcf_yield': 6.445244,
 'fcf_yield_fy1': 5.5641203,
 'yld_on_cost_1y': 5.2065253,
 'yld_on_cost_3y': 5.912495,
 'yld_on_cost_5y': 7.5037518,
 'revenue_change_display_avg_5y': 32.21278,
 'ebit_change_display_avg_5y': 43.7446,
 'eps_change_display_avg_5y': 11.268139,
 'fcf_per_share_change_display_avg_5y': 28.959702,
 'ebitda_change_display_avg_5y': 42.03719,
 'dividend_per_share_change_dislpay_avg_5y': 8.8422165,
 'dividend_lt_fwd_growth_avg_5y': 4.235322,
 'div_grow_rate3_av

In [137]:
# sector_metrics
url = f"https://seekingalpha.com/api/v3/symbols/{STOCK.lower()}/sector_metrics"
querystring = {"filter[fields][]":div_list_seeking_alpha}
response = requests.request("GET", url, headers=headers_div_growth, params=querystring).json()
result = {}
for item in response['data']:
    value = item['attributes']['value']
    metric_id = item['relationships']['metric_type']['data']['id']
    for included in response['included']:
        if included['id'] == metric_id:
            result[included['attributes']['field']] = value
result

{'dividend_lt_fwd_growth': 2.55522,
 'dividend_per_share_change_dislpay': 4.17899,
 'dividend_yield': 4.48653,
 'div_grow_rate10': 3.15015,
 'div_grow_rate3': 0.777233,
 'div_grow_rate5': 1.66106,
 'div_yield_4y': 4.34217,
 'div_yield_fwd': 4.58554,
 'dps_yoy': 5.4298,
 'earnings_yield': 3.47481,
 'earn_yield_gaap_fy1': 3.41031,
 'ebitda_change_display': 9.34481,
 'ebit_change_display': 14.4436,
 'eps_change_display': 10.573,
 'fcf_per_share_change_display': 13.4199,
 'fcf_yield': 7.59678,
 'fcf_yield_fy1': 5.66378,
 'oper_income_fy1_market_cap': 5.22108,
 'oper_income_market_cap': 5.29496,
 'revenue_change_display': 10.1265,
 'yld_on_cost_1y': 3.34395,
 'yld_on_cost_3y': 3.81838,
 'yld_on_cost_5y': 4.1377}

In [138]:
# ticker_metric_grades
url = "https://seekingalpha.com/api/v3/ticker_metric_grades"
querystring = {
    "filter[fields][]":div_list_seeking_alpha,
    "filter[slugs]":f"{STOCK.lower()}",
    "filter[algos][]":["main_quant","dividends"],
    "minified":"false"
}
response = requests.request("GET", url, headers=headers_div_growth, params=querystring).json()
result = {}
for item in response['data']:
    grade = item['attributes']['grade']
    metric_id = item['relationships']['metric_type']['data']['id']
    for included in response['included']:
        if included['id'] == metric_id:
            result[included['attributes']['field']] = grade
result

{'dividend_lt_fwd_growth': 4,
 'dividend_per_share_change_dislpay': 5,
 'dividend_yield': 7,
 'div_grow_rate3': 2,
 'div_yield_4y': 5,
 'div_yield_fwd': 7,
 'earnings_yield': 5,
 'earn_yield_gaap_fy1': 7,
 'ebitda_change_display': 2,
 'ebit_change_display': 2,
 'eps_change_display': 6,
 'fcf_per_share_change_display': 3,
 'revenue_change_display': 2,
 'yld_on_cost_1y': 2,
 'yld_on_cost_3y': 2,
 'yld_on_cost_5y': 1}

In [139]:
# symbol_data
url = "https://seekingalpha.com/api/v3/symbol_data"
querystring = {"fields[]":"dividend_cagrs", "slugs":f"{STOCK.lower()}"}
response = requests.request("GET", url, headers=headers_div_growth, params=querystring).json()
result = response['data'][0]['attributes'] #['dividendCagrs']
result

{'dividendCagrs': {'2021': 8.69565217391304,
  '2020': 9.32607756185055,
  '2019': 8.6346740846674,
  '2018': 15.6848827736974}}

In [140]:
# dividend yield tooltips
url = "https://seekingalpha.com/api/v3/tooltips"

querystring = {"filter[path]" : f"/symbol/{STOCK}"}
response1 = requests.request("GET", url, headers=headers_div_growth, params=querystring).json()

tooltips = {}

for item in response1['data']:
    term = item['attributes']['term']
    content = item['attributes']['content']
    response1[term] = content

querystring = {"filter[path]" : f"/symbol/{STOCK}/dividends/yield"}
response2 = requests.request("GET", url, headers=headers_div_growth, params=querystring).json()

for item in response2['data']:
    term = item['attributes']['term']
    content = item['attributes']['content']
    tooltips[term] = content
    
tooltips

{'Dividend Yield (TTM)': 'The sum of actual ordinary cash dividend payments in the last 12 months divided by the last close price. Special dividends are not included in the yield. Yield (TTM) is calculated for issuers where dividend payment consistency is low, making Yield (FWD) predictions less reliable.',
 'Quant Ranking': 'Quant Ratings are an objective, quantitative view of each stock relative to other stocks in the same sector. The ratings provide an instant summary of the available data about the stock, including the company’s financial results, the stock’s trading history, and sell-side analysts’ estimates of future revenue and earnings. <br><br> The Quant Rankings are based on the underlying rating of each stock and provides the overall rank order of the stock within the sector and overall stock universe. <br><br>For more details, see <a href="/article/4263303-quant-ratings-factor-grades-faq" target="_blank">Quant Ranking and Factor Grades FAQ</a>',
 '4 Year Average Dividend Yi

In [141]:
# articles
url = f"https://seekingalpha.com/api/v3/symbols/{STOCK.lower()}/author_ratings"
querystring = {"include" : ['article']}
response = requests.request("GET", url, headers=headers_div_growth, params=querystring).json()
result = {}
for item in response['data']:
    article_id = item['relationships']['article']['data']['id']
    for item in response['included']:
        if item['id'] == article_id:
            title = item['attributes']['title']
            pro_publish_on = item['attributes']['proPublishOn']
            result[pro_publish_on] = title
result

{'2023-01-11T17:33:00-05:00': 'VICI: More Growth Coming As Blackstone Acquisition Closes',
 '2022-08-11T08:30:00-04:00': 'VICI Properties: What To Do Following This Bull Run',
 '2022-06-09T07:45:00-04:00': 'VICI Properties: Outperformance And A Fat Yield',
 '2022-03-23T08:00:00-04:00': 'What Are The Best REIT Stocks To Buy In 2022? We Like VICI',
 '2023-01-11T15:32:31-05:00': 'VICI Properties: Beware Of A Potential Casino REIT Bubble',
 '2022-09-12T13:49:44-04:00': 'VICI Properties: Overrated And Overvalued',
 '2023-01-06T16:58:26-05:00': 'VICI Properties: A Growth Stock At A Value Price',
 '2022-10-14T08:35:00-04:00': 'VICI Properties: Buy Caesars For The Price Of A Hard Rock',
 '2022-05-01T22:33:36-04:00': 'VICI Properties: Time To Double Down',
 '2022-12-11T17:00:00-05:00': 'My 2022 Top 3 Picks Year-End Report: VICI Value, AbbVie Profits, Google Decision Time',
 '2022-12-07T08:00:00-05:00': 'VICI Hits The Jackpot With MGM Acquisition From Blackstone',
 '2022-12-06T12:37:35-05:00': '

In [142]:
"""quant rating"""
# https://seekingalpha.com/api/v3/symbols/dpz/rating/histories?page[number]=1
# https://seekingalpha.com/api/v3/historical_prices?filter[ticker][slug]=dpz&&filter[as_of_date][gte]=2022-07-25&filter[as_of_date][lte]=2022-09-12&sort=as_of_date
# https://seekingalpha.com/api/v3/tooltips?filter[path]=%2Fsymbol%2FDPZ%2Fratings%2Fquant-ratings
# https://seekingalpha.com/api/v3/symbols/DPZ/rating/periods?filter[periods][]=0&filter[periods][]=3&filter[periods][]=6
# https://seekingalpha.com/api/v3/symbols/dpz/fundamentals_metrics?period_type=annual&statement_type=income-statement&target_currency=USD

'quant rating'

In [143]:
"""wall street ratings"""
# https://seekingalpha.com/api/v3/symbol_data/estimates?estimates_data_items=outperform%2Cbuy%2Chold%2Cunderperform%2Csell&group_by_month=true&period_type=non_periodic&return_window=4&ticker_ids=2557
# https://seekingalpha.com/api/v3/symbol_data/estimates?estimates_data_items=target_price&group_by_month=false&period_type=non_periodic&return_window=4&ticker_ids=2557
# https://seekingalpha.com/api/v3/symbol_data/estimates?estimates_data_items=target_price%2Ctarget_price_high%2Ctarget_price_low&group_by_month=false&period_type=non_periodic&return_window=1&ticker_ids=2557
# https://seekingalpha.com/api/v3/metrics?filter[fields]=authors_rating_strong_buy_count%2Cauthors_rating_buy_count%2Cauthors_rating_hold_count%2Cauthors_rating_sell_count%2Cauthors_rating_strong_sell_count%2Csell_side_rating_strong_buy_count%2Csell_side_rating_buy_count%2Csell_side_rating_hold_count%2Csell_side_rating_sell_count%2Csell_side_rating_strong_sell_count%2Cauthors_rating%2Csell_side_rating&filter[slugs]=dpz&minified=false
# https://seekingalpha.com/api/v3/symbols/DPZ/rating/periods?filter[periods][]=0&filter[periods][]=3&filter[periods][]=6

'wall street ratings'

In [144]:
"""nasdaq analyst"""
# https://www.nasdaq.com/market-activity/stocks/dpz/analyst-research

'nasdaq analyst'

Alpha Spread

In [145]:
"""BeautifulSoup"""
url_ = f"https://www.alphaspread.com/security/nasdaq/aapl/summary"

# Reuse a session object: 
# Instead of creating a new session object for each request, you can reuse the same session object for multiple requests. 
# This can save time by not having to establish a new connection for each request.

# session_object = requests.Session()
# response = session_object.get(url_)

# response = requests.get(url)
# html = response.text
# soup = BeautifulSoup(html, "lxml")
# soup

# https://www.alphaspread.com/security/nasdaq/aapl/summary
# https://www.alphaspread.com/security/nasdaq/aapl/dcf-valuation
# https://www.alphaspread.com/security/nasdaq/aapl/relative-valuation
# https://www.alphaspread.com/security/nasdaq/aapl/analyst-estimates
# https://www.alphaspread.com/security/nasdaq/aapl/profitability
# https://www.alphaspread.com/security/nasdaq/aapl/solvency
# https://www.alphaspread.com/security/nasdaq/aapl/financials/balance-sheet
# https://www.alphaspread.com/security/nasdaq/aapl/financials/income-statement
# https://www.alphaspread.com/security/nasdaq/aapl/financials/cash-flow-statement
# https://www.alphaspread.com/security/nasdaq/aapl/financials/revenue-breakdown
# https://www.alphaspread.com/security/nasdaq/aapl/discount-rate

In [146]:
"""NASDAQ data link"""
# https://docs.data.nasdaq.com/docs/in-depth-usage
# https://data.nasdaq.com/tools/python
f"https://data.nasdaq.com/api/v3/datasets/WIKI/{STOCK}/data.json?api_key={NASDAQ_DATA_LINK_API_KEY}"

"""last date on AAPL is 2018-03-27"""
# url = "https://data.nasdaq.com/api/v3/datasets/WIKI/AAPL/data.json"
# querystring = {"api_key":NASDAQ_DATA_LINK_API_KEY}
# response = requests.request("GET", url, params=querystring).json()
# ndl_df = pd.DataFrame(data=response['dataset_data']['data'], columns=response['dataset_data']['column_names'])
# ndl_df

"""last date on AAPL is 2017-10-31"""
# url = "https://data.nasdaq.com/api/v3/datatables/QUOTEMEDIA/PRICES"
# querystring = {"api_key":NASDAQ_DATA_LINK_API_KEY, 'ticker':'AAPL'}
# response = requests.request("GET", url, params=querystring).json()['datatable']
# ndl_df = pd.DataFrame(data=response['data'], columns=[i['name'] for i in response['columns']])
# ndl_df

"""income from seeking alpha (doesn't work)"""
# url = "https://seekingalpha.com/api/v3/symbols/dpz/fundamentals_metrics"
# querystring = {
#     "period_type" : 'quarterly',
#     'statement_type':'income-statement',
#     'target_currency':'USD'
# }
# response = requests.request("GET", url, headers=headers_div_growth, params=querystring).json()
# response


"income from seeking alpha (doesn't work)"

# Summary

In [147]:
stocks_full_summary = pd.DataFrame()
for x in STOCKS_LIST:
    print(x)
    ticker = yq.Ticker(x)
    # ticker_yf = yf.Ticker(x)

    summary = pd.DataFrame(ticker.summary_detail)
    
    recomendations = ticker.recommendation_trend.reset_index().query('period=="0m"')[['strongBuy','buy','hold','sell','strongSell']].T
    recomendations.columns = [x]
    
    financials = pd.DataFrame(ticker.financial_data)
    key_stats = pd.DataFrame(ticker.key_stats)
    # esg_scores = pd.DataFrame(ticker.esg_scores)
    profile = pd.DataFrame(ticker.summary_profile)

    # earnings
    earnings = ticker.earning_history

    # grades
    grades = ticker.grading_history
    grades['epochGradeDate'] = pd.to_datetime(grades['epochGradeDate'])

    # income
    income_statement = ticker.income_statement().query("periodType=='12M'")

    if 'OperatingIncome' in income_statement.columns:
        income_statement['operatingMargin'] = income_statement['OperatingIncome']/income_statement['TotalRevenue']
    else:
        income_statement['operatingMargin'] = income_statement['PretaxIncome']/income_statement['TotalRevenue']
    
    if 'GrossProfit' in income_statement.columns:
        income_statement['grossMargin'] = income_statement['GrossProfit']/income_statement['TotalRevenue']
    else:
        income_statement['grossMargin'] = np.nan
    income_statement['avgGrossMarginGrowth'] = income_statement['grossMargin'].diff()
    income_statement['avgOperatingMarginGrowth'] = income_statement['operatingMargin'].diff()
    income_statement['avgNetIncomeGrowth'] = income_statement['NetIncome'].diff()/income_statement['NetIncome'].shift(1)
    income_statement['avgTotalRevenueGrowth'] = income_statement['TotalRevenue'].diff()/income_statement['TotalRevenue'].shift(1)

    # balance
    balance = ticker.balance_sheet().query('periodType=="12M"')
    balance['avgTotalAssetsGrowth'] = balance['TotalAssets'].diff()/balance['TotalAssets'].shift(1)
    balance['avgTotalDebtGrowth'] = balance['TotalDebt'].diff()/balance['TotalDebt'].shift(1)

    if 'NetDebt' in balance.columns:
        balance['avgNetDebtGrowth'] = balance['NetDebt'].diff()/balance['NetDebt'].shift(1)
    elif 'CurrentDebt' in balance.columns:
        balance['avgNetDebtGrowth'] = balance['CurrentDebt'].diff()/balance['CurrentDebt'].shift(1)
    else:
        balance['avgNetDebtGrowth'] = np.nan

    # cash flow
    cashflow = ticker.cash_flow().query('periodType=="12M"')
    cashflow['avgFreeCashFlowGrowth'] = cashflow['FreeCashFlow'].diff()/cashflow['FreeCashFlow'].shift(1)

    # final row
    row = pd.concat([profile, summary, key_stats, financials, recomendations], axis=0)
    row = row[~row.index.duplicated(keep='first')].T

    row['longName'] = ticker.price[x]['longName']
    if type(earnings['surprisePercent'][0]) != dict:
        row['avgSurprice'] = earnings['surprisePercent'].median()
        row['lastSurprice'] = earnings.iloc[-1]['surprisePercent']
        row['earning_dynamics'] = earnings.iloc[0]['epsActual'] / earnings.iloc[-1]['epsActual']
    else:
        row['avgSurprice'] = 0
        row['lastSurprice'] = 0
        row['earning_dynamics'] = 0

    row['operatingMargin'] = income_statement.iloc[-1]['operatingMargin']
    row['grossMargin'] = income_statement.iloc[-1]['grossMargin']
    row['operatingMarginStability'] = income_statement['operatingMargin'].max() - income_statement['operatingMargin'].min()
    row['avgGrossMarginGrowth'] = income_statement['avgGrossMarginGrowth'].mean()
    row['avgOperatingMarginGrowth'] = income_statement['avgOperatingMarginGrowth'].mean()
    row['avgNetIncomeGrowth'] = income_statement['avgNetIncomeGrowth'].mean()
    row['avgTotalRevenueGrowth'] = income_statement['avgTotalRevenueGrowth'].mean()
    row['netIncomeStability'] = income_statement['avgNetIncomeGrowth'].max() - income_statement['avgNetIncomeGrowth'].min()

    row['avgTotalAssetsGrowth'] = balance['avgTotalAssetsGrowth'].mean()
    row['avgTotalDebtGrowth'] = balance['avgTotalDebtGrowth'].mean()
    row['avgNetDebtGrowth'] = balance['avgNetDebtGrowth'].mean()
    row['netDebtStability'] = balance['avgNetDebtGrowth'].max() - balance['avgNetDebtGrowth'].min()

    if 'NetDebt' in balance.columns:
        row['lastNetDebt'] = balance.iloc[-1]['NetDebt']
    elif 'CurrentDebt' in balance.columns:
        row['lastNetDebt'] = balance.iloc[-1]['CurrentDebt']
    else:
        row['lastNetDebt'] = np.nan

    row['lastFreeCashFlow'] = cashflow.iloc[-1]['FreeCashFlow']
    row['avgFreeCashFlowGrowth'] = cashflow['avgFreeCashFlowGrowth'].mean()
    row['freeCashFlowStability'] = cashflow['avgFreeCashFlowGrowth'].max() - cashflow['avgFreeCashFlowGrowth'].min()

    row['modeGrade'] = grades.loc[(grades['epochGradeDate'].dt.month == dt.date.today().month)|
        (grades['epochGradeDate'].dt.month == dt.date.today().month-1),'toGrade'].mode()[0]
    row['lastGrade'] = grades.loc[grades['epochGradeDate'] == grades['epochGradeDate'].max(),'toGrade'][0]
    
    stocks_full_summary = pd.concat([stocks_full_summary, row], axis=0)

stocks_full_summary['debtToEBITDA'] = stocks_full_summary['lastNetDebt']/stocks_full_summary['ebitda']
stocks_full_summary.head(5)

AAPL
ABBV
ABR
ABT


KeyboardInterrupt: 

In [ ]:
stocks_full_summary.loc[stocks_full_summary['earningsGrowth'].isna()]

,address1,city,companyOfficers,country,fullTimeEmployees,industry,longBusinessSummary,maxAge,phone,sector,state,website,zip,algorithm,ask,askSize,averageDailyVolume10Day,averageVolume,averageVolume10days,beta,bid,bidSize,coinMarketCapLink,currency,dayHigh,dayLow,dividendRate,dividendYield,exDividendDate,fiftyDayAverage,fiftyTwoWeekHigh,fiftyTwoWeekLow,fiveYearAvgDividendYield,forwardPE,fromCurrency,lastMarket,marketCap,open,payoutRatio,previousClose,priceHint,priceToSalesTrailing12Months,regularMarketDayHigh,regularMarketDayLow,regularMarketOpen,regularMarketPreviousClose,regularMarketVolume,toCurrency,tradeable,trailingAnnualDividendRate,trailingAnnualDividendYield,trailingPE,twoHundredDayAverage,volume,52WeekChange,SandP52WeekChange,bookValue,category,dateShortInterest,earningsQuarterlyGrowth,enterpriseToEbitda,enterpriseToRevenue,enterpriseValue,floatShares,forwardEps,fundFamily,heldPercentInsiders,heldPercentInstitutions,impliedSharesOutstanding,lastDividendDate,lastDividendValue,lastFiscalYearEnd,lastSplitDate,lastSplitFactor,legalType,mostRecentQuarter,netIncomeToCommon,nextFiscalYearEnd,pegRatio,priceToBook,profitMargins,sharesOutstanding,sharesPercentSharesOut,sharesShort,sharesShortPreviousMonthDate,sharesShortPriorMonth,shortPercentOfFloat,shortRatio,trailingEps,currentPrice,currentRatio,debtToEquity,earningsGrowth,ebitda,ebitdaMargins,financialCurrency,freeCashflow,grossMargins,grossProfits,numberOfAnalystOpinions,operatingCashflow,operatingMargins,quickRatio,recommendationKey,recommendationMean,returnOnAssets,returnOnEquity,revenueGrowth,revenuePerShare,targetHighPrice,targetLowPrice,targetMeanPrice,targetMedianPrice,totalCash,totalCashPerShare,totalDebt,totalRevenue,strongBuy,buy,hold,sell,strongSell,longName,avgSurprice,lastSurprice,earning_dynamics,operatingMargin,grossMargin,operatingMarginStability,avgGrossMarginGrowth,avgOperatingMarginGrowth,avgNetIncomeGrowth,avgTotalRevenueGrowth,netIncomeStability,avgTotalAssetsGrowth,avgTotalDebtGrowth,avgNetDebtGrowth,netDebtStability,lastNetDebt,lastFreeCashFlow,avgFreeCashFlowGrowth,freeCashFlowStability,modeGrade,lastGrade,address2,fax,debtToEBITDA,TOTAL_SCORE
BRK-B,3555 Farnam Street,Omaha,[],United States,372000,Insurance—Diversified,"Berkshire Hathaway Inc., through its subsidiar...",86400,402 346 1400,Financial Services,NE,https://www.berkshirehathaway.com,68131,None,306.79,800,4361740,4174663,4361740,0.919899,306.56,900,None,USD,307.82,304.32,NaN,NaN,NaN,298.4184,362.1,259.85,NaN,20.094444,None,None,678343606272,304.38,0.0,302.0,2,2.293856,307.82,304.32,304.38,302.0,646682,None,False,0.0,0.0,59.01056,301.9704,646682,0.02578,-0.186294,310560.2,None,2022-11-30 01:00:00,NaN,1.638,0.055,16294445056,1205911,15.3,None,0.00441,0.65289,0,NaN,NaN,2021-12-31 01:00:00,2010-01-21 01:00:00,50:1,None,2022-09-30 02:00:00,-1336999936,2023-12-31 01:00:00,0.89,0.00099,-0.00452,1304380032,0.0024,5185763,2022-10-31 01:00:00,6398341,0.0045,1.32,5.21,307.445,1.459,25.12,NaN,9946999808,0.03364,USD,-23768750080,-0.00304,114863000000,1,34817998848,-0.00304,1.245,hold,2.7,-0.00062,-0.00102,0.09,200580.47,362.0,362.0,362.0,362.0,108957999104,74383.305,116495998976,295721992192,1,3,2,0,0,Berkshire Hathaway Inc.,0.2000,0.322,0.926346,0.314931,NaN,0.297179,NaN,0.099060,6.627330,0.188514,19.725686,0.107050,0.056210,-0.093199,1.379631,2.607800e+10,2.614500e+10,0.049562,0.201502,Equal-Weight,Equal-Weight,NaN,402 346 3375,2.621695,NaN
MO,6601 West Broad Street,Richmond,[],United States,6000,Tobacco,"Altria Group, Inc., through its subsidiaries, ...",86400,804 274 2200,Consumer Defensive,VA,https://www.altria.com,23230,None,45.59,1000,8591000,8564788,8591000,0.67852,45.58,800,None,USD,45.76,45.4009,3.76,0.0808,2022-12-21 01:00:00,45.5256,57.05,40.35,6.83,9.034862,None,None,81607720960,45.61,1.4163,45.5,2,3.944117,45.76,45.4009,45.61,45.5,1797544,None,False,3.64,0.08,17.718172,47.3383,1797544,-0.009386,-0.186294,-2.359,None,2022-11-30 01:00:00,NaN,8.631,5.183,107251556352,1790326680,5.04,None,0.00

In [ ]:
stocks_full_summary[['grossMargins','grossMargin']]

,grossMargins,grossMargin
AAPL,0.4331,0.433096
ABBV,0.69832,0.689556
ABR,0.91626,NaN
ABT,0.58163,0.569658
AFL,0.42594,NaN
AMD,0.5095,0.482475
AMGN,0.7578,0.751569
AMZN,0.43044,0.141149
ASML,0.51142,0.527054
ATVI,0.71129,0.736794


In [ ]:
# stocks_full_summary['TOTAL_SCORE'] = \
#     stocks_full_summary['marketCap']\
#     *stocks_full_summary['dividendYield']\
#     *stocks_full_summary['avgNetIncomeGrowth']\
#     *stocks_full_summary['avgFreeCashFlowGrowth']\
#     *stocks_full_summary['operatingMargins']*stocks_full_summary['operatingMargin']*stocks_full_summary['avgOperatingMarginGrowth']\
#     *stocks_full_summary['grossMargins']*stocks_full_summary['grossMargin']*stocks_full_summary['avgGrossMarginGrowth']\
#     *(stocks_full_summary['forwardEps'] / stocks_full_summary['trailingEps'])\
#     *stocks_full_summary['avgSurprice']\
#     *stocks_full_summary['totalCashPerShare']\
#     *stocks_full_summary['payoutRatio']
    # stocks_full_summary['freeCashflow'] * stocks_full_summary['lastFreeCashFlow'] * stocks_full_summary['avgFreeCashFlowGrowth']\
    # *stocks_full_summary['operatingCashflow']\
    # *stocks_full_summary['operatingMargins']*stocks_full_summary['operatingMargin']*stocks_full_summary['avgOperatingMarginGrowth']\
    # *stocks_full_summary['grossMargins']*stocks_full_summary['grossMargin']*stocks_full_summary['avgGrossMarginGrowth']\
    # *stocks_full_summary['dividendYield'] * stocks_full_summary['fiveYearAvgDividendYield'] * stocks_full_summary['trailingAnnualDividendYield']\
    # *(stocks_full_summary['forwardPE'] / stocks_full_summary['trailingPE'])\
    # *(stocks_full_summary['forwardEps'] / stocks_full_summary['trailingEps'])\
    # /(stocks_full_summary['avgNetDebtGrowth'] * stocks_full_summary['debtToEquity'] * stocks_full_summary['debtToEBITDA'])\
    # *(stocks_full_summary['returnOnAssets']*stocks_full_summary['returnOnEquity'])\
    # *(stocks_full_summary['revenueGrowth']*stocks_full_summary['revenuePerShare'])\
    # *(stocks_full_summary['ebitda']*stocks_full_summary['ebitdaMargins'])\
    # *(stocks_full_summary['avgNetIncomeGrowth'] * stocks_full_summary['avgTotalRevenueGrowth'])\
    # *(stocks_full_summary['avgTotalAssetsGrowth'] - stocks_full_summary['avgTotalDebtGrowth'])\
    # *stocks_full_summary['avgSurprice']\
    # *stocks_full_summary['totalCashPerShare']\
    # *stocks_full_summary['marketCap']\
    # *stocks_full_summary['payoutRatio']
stocks_full_summary[['longName','marketCap']].sort_values(by='marketCap', ascending=False)

,longName,marketCap
AAPL,Apple Inc.,2159683567616
MSFT,Microsoft Corporation,1817474367488
GOOGL,Alphabet Inc.,1162320281600
AMZN,"Amazon.com, Inc.",884334395392
BRK-B,Berkshire Hathaway Inc.,678343606272
JNJ,Johnson & Johnson,464148594688
XOM,Exxon Mobil Corporation,446443225088
TSLA,"Tesla, Inc.",438737797120
V,Visa Inc.,438432038912
NVDA,NVIDIA Corporation,411329527808


In [ ]:
isna_stocks = stocks_full_summary.isna().sum()
isna_stocks

address1              0
city                  0
companyOfficers       0
country               0
fullTimeEmployees     0
                     ..
lastGrade             0
address2             43
fax                  46
debtToEBITDA         11
TOTAL_SCORE          13
Length: 148, dtype: int64

In [ ]:
# cashflow = ticker.cash_flow().query('periodType=="12M"')
# cashflow.FreeCashFlow

In [ ]:
stocks_summary = stocks_full_summary[[
    'longName', 
    'sector',
    'industry',
    'marketCap',
    'currentPrice',
    'averageVolume',
    'earningsGrowth',
    'revenueGrowth',
    'trailingPE',
    'forwardPE',
    'priceToSalesTrailing12Months',
    'dividendYield',
    'fiveYearAvgDividendYield',
    'trailingAnnualDividendYield',
    'exDividendDate',
    'freeCashflow',
    'revenuePerShare',
    'targetHighPrice',
    'targetLowPrice',
    'targetMeanPrice',
    'targetMedianPrice',
    'recommendationKey',
    'modeGrade',
    'strongBuy','buy','hold','sell','strongSell'
    ]].copy()


stocks_summary['exDividendDate'] = pd.to_datetime(stocks_summary['exDividendDate']).dt.strftime('%Y-%m-%d')

stocks_summary['marketCap'] = stocks_summary['marketCap']/1e9
stocks_summary['freeCashflow'] = stocks_summary['marketCap']/1e9
stocks_summary['averageVolume'] = stocks_summary['averageVolume']/1e6
stocks_summary['targetHighPrice'] = stocks_summary['targetHighPrice']/stocks_summary['currentPrice']-1
stocks_summary['targetLowPrice'] = stocks_summary['targetLowPrice']/stocks_summary['currentPrice']-1
stocks_summary['targetMeanPrice'] = stocks_summary['targetMeanPrice']/stocks_summary['currentPrice']-1
stocks_summary['targetMedianPrice'] = stocks_summary['targetMedianPrice']/stocks_summary['currentPrice']-1
stocks_summary

,longName,sector,industry,marketCap,currentPrice,averageVolume,earningsGrowth,revenueGrowth,trailingPE,forwardPE,priceToSalesTrailing12Months,dividendYield,fiveYearAvgDividendYield,trailingAnnualDividendYield,exDividendDate,freeCashflow,revenuePerShare,targetHighPrice,targetLowPrice,targetMeanPrice,targetMedianPrice,recommendationKey,modeGrade,strongBuy,buy,hold,sell,strongSell
AAPL,Apple Inc.,Technology,Consumer Electronics,2159.683568,135.76,87.201515,0.048,0.081,22.21931,20.053175,5.476871,0.007,0.99,0.006803,2022-11-04,0.000002,24.317,0.576311,-0.101355,0.29972,0.296405,buy,Buy,11,21,6,0,0
ABBV,AbbVie Inc.,Healthcare,Drug Manufacturers—General,286.396514,161.945,5.679307,0.242,0.033,21.592669,13.90086,4.953329,0.0367,4.35,0.035123,2023-01-12,0.0,32.638,0.234987,-0.166384,-0.006576,-0.01201,buy,Buy,4,6,10,0,0
ABR,"Arbor Realty Trust, Inc.",Real Estate,REIT—Mortgage,2.343018,13.66,2.206565,-0.285,-0.054,7.189474,9.046357,3.608657,0.119,9.12,0.114925,2022-11-17,0.0,4.091,0.354319,0.061493,0.207906,0.24451,buy,Market Outperform,0,2,1,0,0
ABT,Abbott Laboratories,Healthcare,Medical Devices,188.20096,107.94,5.985533,-0.308,-0.047,24.36569,24.476192,4.179457,0.0191,1.53,0.017654,2023-01-12,0.0,25.592,0.334074,-0.166203,0.082824,0.088568,buy,Outperform,7,6,7,0,0
AFL,Aflac Incorporated,Financial Services,Insurance—Life,44.669317,71.84,2.715812,0.917,-0.08,9.1867,13.157509,2.134428,0.0244,2.35,0.021653,2023-02-14,0.0,32.465,0.113586,-0.24833,-0.056654,-0.039532,hold,Equal-Weight,1,2,10,1,1
AMD,"Advanced Micro Devices, Inc.",Technology,Semiconductors,108.592439,67.34,80.917463,-0.947,0.29,41.8323,18.452053,4.756985,NaN,NaN,0.0,1995-04-27,0.0,15.646,1.970003,-0.108999,0.349421,0.299376,buy,Buy,5,6,15,4,1
AMGN,Amgen Inc.,Healthcare,Drug Manufacturers—General,142.153449,266.415,2.890631,0.202,-0.008,21.381622,14.424202,5.398916,0.0322,2.82,0.028631,2023-02-14,0.0,48.29,0.219901,-0.324362,0.00092,0.015333,hold,Overweight,3,7,15,0,0
AMZN,"Amazon.com, Inc.",Consumer Cyclical,Internet Retail,884.334395,86.685,79.831473,-0.097,0.147,79.52752,52.21988,1.760952,NaN,NaN,0.0,NaN,0.000001,49.349,1.685009,-0.077118,0.613774,0.580435,buy,Buy,15,28,3,1,0
ASML,ASML Holding N.V.,Technology,Semiconductor Equipment & Materials,238.440071,576.9219,1.402849,0.007,0.102,38.90235,29.13747,12.085888,0.0116,0.83,0.009003,2022-11-03,0.0,49.224,0.565567,-0.181865,0.196678,0.148232,buy,Overweight,2,3,2,0,0
ATVI,"Activision Blizzard, Inc.",Communication Services,Electronic Gaming & Multimedia,59.193844,75.635,6.865368,-0.329,-0.139,35.509388,19.443445,8.045921,0.0062,0.59,0.006194,2022-04-13,0.0,9.432,0.322139,0.031269,0.220202,0.256032,buy,Buy,9,11,7,0,0


In [ ]:
# yq.Ticker('AAPL').earnings
# pd.DataFrame(yq.Ticker('AAPL').all_modules)

funds

In [ ]:
yq.Ticker('SCHD').fund_performance

{'SCHD': {'maxAge': 1,
  'performanceOverview': {'asOfDate': '2022-12-14 01:00:00',
   'ytdReturnPct': -0.0095,
   'oneYearTotalReturn': 0.026700001,
   'threeYearTotalReturn': 0.1433},
  'performanceOverviewCat': {'ytdReturnPct': 0.109,
   'fiveYrAvgReturnPct': 0.15359999},
  'trailingReturns': {'asOfDate': '2022-12-12 01:00:00',
   'ytd': -0.0138792,
   'oneMonth': 0.0096464,
   'threeMonth': 0.051581603,
   'oneYear': 0.0182523,
   'threeYear': 0.1406709,
   'fiveYear': 0.13403301,
   'tenYear': 0.13838139,
   'lastBullMkt': 0.0,
   'lastBearMkt': 0.0},
  'trailingReturnsNav': {'ytd': -0.0138792,
   'oneMonth': 0.0096464,
   'threeMonth': 0.051581603,
   'oneYear': 0.0182523,
   'threeYear': 0.1406709,
   'fiveYear': 0.13403301,
   'tenYear': 0.13838139},
  'trailingReturnsCat': {'ytd': 0.109,
   'oneMonth': -0.001,
   'threeMonth': 0.0326,
   'oneYear': 0.17040001,
   'threeYear': 0.105299994,
   'fiveYear': 0.15359999,
   'tenYear': 0.0655,
   'lastBullMkt': 0.0,
   'lastBearMkt':

In [ ]:
yq.Ticker('SCHD').fund_profile

{'SCHD': {'maxAge': 1,
  'styleBoxUrl': 'https://s.yimg.com/lq/i/fi/3_0stylelargeeq1.gif',
  'family': 'Schwab ETFs',
  'categoryName': 'Large Value',
  'legalType': 'Exchange Traded Fund',
  'managementInfo': {'managerName': None, 'managerBio': None},
  'feesExpensesInvestment': {'annualReportExpenseRatio': 0.00059999997,
   'annualHoldingsTurnover': 0.14,
   'totalNetAssets': 75870.54,
   'projectionValues': {}},
  'feesExpensesInvestmentCat': {'annualReportExpenseRatio': 0.0034,
   'annualHoldingsTurnover': 34.69,
   'totalNetAssets': 75870.54,
   'projectionValuesCat': {}},
  'brokerages': []}}

In [ ]:
yq.Ticker('SCHD').fund_sector_weightings

,SCHD
0,
realestate,0.0000
consumer_cyclical,0.0747
basic_materials,0.0196
consumer_defensive,0.1339
technology,0.1633
communication_services,0.0487
financial_services,0.2036
utilities,0.0037
industrials,0.1640


In [ ]:
yq.Ticker('SCHD').fund_top_holdings

symbol                           holdingName  holdingPercent
symbol row                                                             
SCHD   0      MRK                        Merck & Co Inc          0.0457
       1     AMGN                             Amgen Inc          0.0433
       2      IBM  International Business Machines Corp          0.0425
       3     AVGO                          Broadcom Inc          0.0415
       4      PEP                           PepsiCo Inc          0.0411
       5      BLK                         BlackRock Inc          0.0405
       6      LMT                  Lockheed Martin Corp          0.0405
       7       VZ            Verizon Communications Inc          0.0396
       8     CSCO                     Cisco Systems Inc          0.0396
       9      PFE                            Pfizer Inc          0.0396

In [ ]:
yq.Ticker('SPHD').fund_top_holdings

symbol                      holdingName  holdingPercent
symbol row                                                        
SPHD   0      IRM                Iron Mountain Inc          0.0321
       1       MO                 Altria Group Inc          0.0314
       2      PPL                         PPL Corp          0.0308
       3      WMB           Williams Companies Inc          0.0291
       4        T                         AT&T Inc          0.0289
       5      KMI        Kinder Morgan Inc Class P          0.0272
       6       PM  Philip Morris International Inc          0.0253
       7      CVX                     Chevron Corp          0.0244
       8      PRU         Prudential Financial Inc          0.0234
       9     AMCR        Amcor PLC Ordinary Shares          0.0232

In [ ]:
yq.Ticker('SCHD').price

{'SCHD': {'maxAge': 1,
  'preMarketChangePercent': -0.0105154,
  'preMarketChange': -0.809998,
  'preMarketTime': '2022-12-15 15:29:58',
  'preMarketPrice': 76.22,
  'preMarketSource': 'FREE_REALTIME',
  'postMarketChangePercent': -0.00039754115,
  'postMarketChange': -0.030006409,
  'postMarketTime': 1671143702,
  'postMarketPrice': 75.45,
  'postMarketSource': 'FREE_REALTIME',
  'regularMarketChangePercent': -0.02012197,
  'regularMarketChange': -1.5499954,
  'regularMarketTime': '2022-12-15 22:00:00',
  'priceHint': 2,
  'regularMarketPrice': 75.48,
  'regularMarketDayHigh': 76.4299,
  'regularMarketDayLow': 75.1401,
  'regularMarketVolume': 3815032,
  'regularMarketPreviousClose': 77.03,
  'regularMarketSource': 'DELAYED',
  'regularMarketOpen': 76.34,
  'exchange': 'PCX',
  'exchangeName': 'NYSEArca',
  'exchangeDataDelayedBy': 0,
  'marketState': 'POST',
  'quoteType': 'ETF',
  'symbol': 'SCHD',
  'underlyingSymbol': None,
  'shortName': 'Schwab US Dividend Equity ETF',
  'longNa

In [ ]:
yq.Ticker('SCHD').key_stats

{'SCHD': {'maxAge': 1,
  'priceHint': 2,
  'category': 'Large Value',
  'ytdReturn': -0.0095,
  'beta3Year': 0.89,
  'totalAssets': 44960911360,
  'yield': 0.031400003,
  'fundFamily': 'Schwab ETFs',
  'fundInceptionDate': '2011-10-20 02:00:00',
  'legalType': 'Exchange Traded Fund',
  'threeYearAverageReturn': 0.1433,
  'fiveYearAverageReturn': 0.1245,
  'lastSplitFactor': None}}

In [ ]:
pd.DataFrame(yq.Ticker('SCHD').all_modules)

,SCHD
assetProfile,"{'phone': 'NA', 'longBusinessSummary': 'To pur..."
defaultKeyStatistics,"{'maxAge': 1, 'priceHint': 2, 'category': 'Lar..."
fundPerformance,"{'maxAge': 1, 'performanceOverview': {'asOfDat..."
fundProfile,"{'maxAge': 1, 'styleBoxUrl': 'https://s.yimg.c..."
pageViews,"{'shortTermTrend': 'DOWN', 'midTermTrend': 'UP..."
price,"{'maxAge': 1, 'preMarketSource': 'FREE_REALTIM..."
quoteType,"{'exchange': 'PCX', 'quoteType': 'ETF', 'symbo..."
summaryDetail,"{'maxAge': 1, 'priceHint': 2, 'previousClose':..."
summaryProfile,"{'phone': 'NA', 'longBusinessSummary': 'To pur..."
topHoldings,"{'maxAge': 1, 'stockPosition': 0.9995, 'bondPo..."


screener

In [ ]:
s = yq.Screener()
s.available_screeners
# s.get_screeners('all_cryptocurrencies_us')

['accident_health_insurance',
 'advertising_agencies',
 'aerospace_defense_major_diversified',
 'aerospace_defense_products_services',
 'aggressive_small_caps',
 'agricultural_chemicals',
 'air_delivery_freight_services',
 'air_services_other',
 'all_cryptocurrencies_au',
 'all_cryptocurrencies_ca',
 'all_cryptocurrencies_eu',
 'all_cryptocurrencies_gb',
 'all_cryptocurrencies_in',
 'all_cryptocurrencies_us',
 'aluminum',
 'apparel_stores',
 'appliances',
 'application_software',
 'asset_management',
 'auto_dealerships',
 'auto_manufacturers_major',
 'auto_parts',
 'auto_parts_stores',
 'auto_parts_wholesale',
 'basic_materials',
 'basic_materials_wholesale',
 'beverages_brewers',
 'beverages_soft_drinks',
 'beverages_wineries_distillers',
 'biotechnology',
 'broadcasting_radio',
 'broadcasting_tv',
 'building_materials_wholesale',
 'business_equipment',
 'business_services',
 'business_software_services',
 'catalog_mail_order_houses',
 'catv_systems',
 'cement',
 'chemicals_major_dive